In [ ]:
# Importing libraries
import os
import logging
import numpy as np
import pandas as pd
import torch
from torch import cuda
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import os

# Importing the mT5 modules from huggingface/transformers
from transformers import T5Tokenizer, MT5Model, MT5Config, MT5ForConditionalGeneration, AutoModelForSeq2SeqLM

In [ ]:
rng = 42

# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(rng)
np.random.seed(rng) 
torch.backends.cudnn.deterministic = True

In [ ]:
path = '/data/imeza/text_datasets/outputs_mT5/model_files/4'

#config = MT5Config.from_pretrained(path)
#model = MT5ForConditionalGeneration.from_pretrained(path)
model = AutoModelForSeq2SeqLM.from_pretrained(path)
tokenizer = T5Tokenizer.from_pretrained(path)

In [ ]:
text = """
La Secretaria Ejecutiva de Fundación PIDEE (Fundación de Protección a la Infancia Dañada por los Estados de Emergencia) Vivian Murúa, afirmó que al día de hoy existen transgresiones a los derechos de niños, niñas y adolescentes (NNA) y que la ley de garantías y protección integral de los derechos de la niñez y adolescencia, no está en su plena aplicación ya que sólo entró en vigencia en marzo de este año.
En el marco del Día Internacional de los Derechos Humanos, Murúa afirmó que “hay transgresiones en todas esas áreas, sobre todo para NNA. Tengamos presente que Chile recién tenemos una Ley de Garantías ahora en marzo de 2022, a pesar que el país había ratificado la Convención sobre los Derechos del Niño en el año 1990. Recién en marzo de este año se promulga la Ley sobre garantía y protección integral de los niños, niñas y adolescentes. Ahora, esa ley hay que ver cómo se va a hacer carne en todos los ministerios y en todas las fuerzas sociales y estatales que tiene que ver con garantizar los derechos de la niñez. Pero hasta la fecha vemos que no hay garantías reales”.
Indicó que como fundación ligada a la promoción y protección de los Derechos Humanos de NNA, “seguimos con causas de jóvenes que fueron agredidos violentamente por agentes del Estado durante la revuelta y están casi todos detenidos, están  las causas pendientes, pero Carabineros de Chile ha sido constante en no presentar evidencias, en no ir y ha sido muy lento el proceso, pero seguimos adelante”.
Murúa recordó que el PIDEE desde el 2019 abrió un área jurídica en la que se entrega apoyo a familias cuyos hijos e hijas han sido vulnerados por agentes del Estado desde el estallido social. Entre los casos más relevantes está el de Geraldine Alvarado, adolescente que recibió el impacto de una bomba lacrimógena en su cabeza el 10 de Diciembre de 2019, quien estuvo al borde de la muerte y que aún tiene secuelas tras el impacto.
La joven “ya cumplió sus 18 años y en este momento se está viendo la posibilidad de recibir indemnización a través del INDH, pero no está claro qué sucede ahí. Ha sido difícil para Geraldine poder integrarse a la sociedad, tuvo que aprender a caminar y a hablar, con lo que eso significa en una ciudad con todos los problemas físicos (…) ha sido un proceso difícil, pero tiene el apoyo de su papá, del PIDEE y es un caso que está pendiente y aún no hay una solución, búsqueda y castigo de los culpables”, señaló la profesional.
Finalmente, Murúa indicó que el Día Internacional por los Derechos Humanos, es “importante para poder rescatar y poner en relieve lo que son los derechos humanos. En el caso de la niñez, nosotros en este momento estamos iniciando un trabajo formativo en dos comunas de Santiago en el tema de los DDHH. Vamos a trabajar con niños, profesores y dirigentes sociales en la idea de forma poder formar garantes ligados a la defensa y protección de los derechos humanos, principalmente ligado a los NNA”.
La representante recordó que el “PIDEE nace el año 1979 ante la demanda de las familias afectadas por las violaciones de los Derechos Humanos durante la dictadura militar para entregar asistencia a las y los niños afectados por trastornos derivados de la situación que vivían en las áreas de salud mental y física, educacional y cultural, además de satisfacer sus necesidades básicas de sobrevivencia”.
Murúa agregó que “desde la creación de la Fundación PIDEE se llevó un registro de la atención a niños, niñas, adolescentes y sus familias. Estos registros constan de relatos directos, testimonios, dibujos e información de prensa que complementa y certifica, además, la situación represiva vivida por cada niño, niña y su familia”, destaca el organismo.
Añade que “en el año 2003 la UNESCO inscribió los archivos de la Fundación PIDEE en el registro Memoria del Mundo, tras una sistematización de la documentación generada en atención y asistencia de la infancia. La información fue digitalizada y destinada al Museo de la Memoria de los Derechos Humanos, incluyendo documentos originales”.
"""

tokenized_text = tokenizer.encode(text, return_tensors="pt")

summary_ids = model.generate(
    tokenized_text,
    max_length=150, 
    num_beams=2,
    repetition_penalty=2.5, 
    length_penalty=2.0, 
    early_stopping=True
)

output = tokenizer.decode(summary_ids[0], 
                          skip_special_tokens=True).replace("<extra_id_0>", "")

print(output)

In [ ]:
df = pd.read_parquet(
        "/data/imeza/text_datasets/data_summarization_with_title.parquet"
    )

In [ ]:
(df.title == '').sum()